# Домашнее задание № 7

### Задание 1 Реализовать алгоритм Леска и проверить его на реальном датасете (8 баллов)

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

Реализуйте его

In [8]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

In [9]:
lemmatizer = WordNetLemmatizer()

In [10]:
def get_words_in_context(text, window=5):
    word2context = []
    text_tokenized = [word.lower() for word in word_tokenize(text)]
    for i, word in enumerate(text_tokenized):
        word2context.append((word, text_tokenized[max(0, i-window):i] + text_tokenized[i+1:i+window]))
        
    return word2context

In [308]:
def lesk(word, sentence):
    bestsense = 0
    maxoverlap = 0
    lemmatized = [lemmatizer.lemmatize(word) for word in sentence]
    synsets = wn.synsets(word)
    
    for i, syns in enumerate(synsets):
        definition = syns.definition().lower().split()
        def_lemmatized = [lemmatizer.lemmatize(word) for word in definition]
        
        overlap = len(set(lemmatized) & set(def_lemmatized))
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i

    return bestsense

**Проверьте насколько хорошо работает такой метод на датасете из семинара**

In [12]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

**Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).**

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу)

In [309]:
all_words = 0
same_words = 0
for sentence in corpus_wsd:
    tokens = [word[2].lower() for word in sentence if len(word) == 3]
    for i, word in enumerate(sentence):
        if word[0]:
            all_words += 1
            context = tokens[max(0, i-5):i] + tokens[i+1:i+5]
            meaning = lesk(word[1], context)
            if wn.synsets(word[1])[meaning] == wn.lemma_from_key(word[0]).synset():
                same_words += 1
print('Accuracy score:', same_words/all_words)

Accuracy score: 0.37558198180173646


### Задание 2* (2 балла)

В семинаре для WSI на данных Диалога использовался только Fastext. Попробуйте заменить его на адаграм (обучите свою модель или используйте предобученную out.pkl или https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib), а также поэкспериментируйте с разными алгоритмами кластеризации и их параметрами (можно использовать только те алгоритмы, которые обучаются достаточно быстро)

Для каждого эксперимента рассчитайте ARI

In [177]:
import pandas as pd
import numpy as np
import adagram
from razdel import tokenize
from pymorphy2 import MorphAnalyzer
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering, AffinityPropagation
from sklearn.metrics import adjusted_rand_score
from nltk.corpus import stopwords
from string import punctuation
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [6]:
morph = MorphAnalyzer()

In [310]:
df = pd.read_csv('train.csv', sep='\t')

In [78]:
ada = adagram.VectorModel.load("out.pkl")

In [94]:
def normalize(text):
    words = [token.text.strip(punct) for token in list(tokenize(text))]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]
    return words

In [98]:
def get_embedding_adagram(text, model, window, dim):
    
    
    word2context = []
    for i in range(len(text)-1):
        left = max(0, i-window)
        word = text[i]
        left_context = text[left:i]
        right_context = text[i+1:i+window]
        context = left_context + right_context
        word2context.append((word, context))

    vectors = np.zeros((len(word2context), dim))
    
    for i,word in enumerate(word2context):
        word, context = word
        try:
            sense = model.disambiguate(word, context).argmax()
            v = model.sense_vector(word, sense)
            vectors[i] = v 

        except (KeyError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
        
    return vector

In [90]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [232]:
def clusterize(model, window):
    ARI = []

    for key, _ in grouped_df:
        texts = grouped_df.get_group(key)['context'].apply(normalize)
        X = np.zeros((len(texts), 100))

        for i, text in enumerate(texts):
            text = [word for word in text if word != key]
            X[i] = get_embedding_adagram(text, ada, window, 100)

        model.fit(X)
        labels = np.array(model.labels_) + 1
        
        ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))

    print(np.mean(ARI))

#### KMeans

In [233]:
km = KMeans(n_clusters = 2, max_iter=1000, n_init=2)

In [235]:
clusterize(km, 4)

C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


0.29596226533895065


#### SpectralClustering

In [264]:
sc = SpectralClustering(n_clusters=5, n_components = 4, n_neighbors = 12)

In [265]:
clusterize(sc, 4)

C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


0.21284506922564658


#### AgglomerativeClustering

In [287]:
ac = AgglomerativeClustering()

In [288]:
clusterize(ac, 4)

C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


0.36738950708833995


#### AffinityPropagation

In [306]:
ap = AffinityPropagation(damping = 0.7, preference=-6)

In [307]:
clusterize(ap, 5)

C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
C:\Users\AFL\anaconda3\lib\site-packages\sklearn\cluster\_affinity_propagation.py:148: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 1.0 (renaming of 0.25) which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  warnings.warn(
C:\Users\AFL\anaconda3\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
C:\Users\AFL\anaconda3\lib\site-packages\sklearn\cluster\_affinity_propagation.py:148: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 1.0 (renaming of 0.25) which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of version

0.2441651885851499


C:\Users\AFL\anaconda3\lib\site-packages\sklearn\cluster\_affinity_propagation.py:148: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 1.0 (renaming of 0.25) which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  warnings.warn(


Вывод: лучший ARI показала модель AgglomerativeClustering с дефолтными параметрами и размером окна контекста 4.